In [1]:
import os, time, copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from PIL import Image

from train import *
from utils import *
from dataset import *
from metrics import *
from FCN.fcn16 import *
# from FCN.fcn32_alex import *

In [2]:
set_seed(1)
os.environ["CUDA_VISIBLE_DEVICES"] = '2'


In [3]:
# Load data
train_data = VOC(mode = 'train', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
train_loader = data.DataLoader(train_data, batch_size=1, num_workers = 0, shuffle=True)

val_data = VOC(mode = 'val', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
val_loader = data.DataLoader(val_data, batch_size=1, num_workers = 0, shuffle=True)

In [4]:
def train(model, device, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0

    loss_sum = torch.zeros(1)
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    optimizer.zero_grad()
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
            
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc += pixel_accuracy(out_mask, target_mask)/20
        macc += mean_accuracy(out_mask, target_mask)/20
        mIU += mean_IU(out_mask, target_mask)/20
        fIU += frequency_weighted_IU(out_mask, target_mask)/20
        
        #calculate loss
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)/20
        loss.backward()
        loss_sum += loss
        
        # Implement mini-batch by accumulate 
        # update gradient every 20 image
        if i%20 == 19 or i == 1111:
            optimizer.step()
            print("Batch %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i//20, loss_sum, pacc, macc, mIU, fIU))
            
            running_loss += loss_sum.item() * 20
            r_pacc += pacc * 20
            r_macc += macc * 20
            r_mIU += mIU * 20
            r_fIU += fIU * 20
            
            # batch initialize
            loss_sum, pacc, macc, mIU, fIU = 0, 0, 0, 0, 0
            optimizer.zero_grad()
            
            # show output mask
#             img = decode_segmap(out_mask)
#             plt.imshow(img); plt.axis('off'); plt.show()
        
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1112 for m in epoch_metric]
    
    print('Train:\t%s' %epoch_metric)
    return epoch_metric

In [5]:
def validate(model, device, val_loader, criterion, mode = 'val'):
    # switch to evaluate mode
    model.eval()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc = pixel_accuracy(out_mask, target_mask)
        macc = mean_accuracy(out_mask, target_mask)
        mIU = mean_IU(out_mask, target_mask)
        fIU = frequency_weighted_IU(out_mask, target_mask)
        
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)
        
        if i % 100 == 99:
            print("Val %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i, loss, pacc, macc, mIU, fIU))
        # statistics
        running_loss += loss.item()
        r_pacc += pacc
        r_macc += macc
        r_mIU += mIU
        r_fIU += fIU

    #epoch_loss = running_loss / 1111
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1111 for m in epoch_metric]
    
    if mode == 'val':
        print('Val metric:\t%s' %epoch_metric)
    else:
        print('Test metric:\t%s' %epoch_metric)
    
    return epoch_metric  #, copy.deepcopy(model.state_dict())

In [6]:
def train_model(model, device, train_loader, val_loader, lr = 1e-4, dict_name = 'fcn32', num_epochs=50):
    
    criterion = nn.CrossEntropyLoss(ignore_index = 255)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5**(-4))

    if device == 'cuda':
        model.to(device)
        cudnn.benchmark = True
        criterion = criterion.cuda()
    
    since = time.time()
    #M = {'train' : [], 'val' : []}
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        
        train_metric = train(model, device, train_loader, criterion, optimizer)
        M['train'].append(train_metric)
        
        # evaluate on validation set
        val_metric = validate(model, device, val_loader, criterion)
        M['val'].append(val_metric)
        
        if val_metric[1] > best_acc:
            best_acc = val_metric[1]
            best_model_wts = copy.deepcopy(model.state_dict())
            SaveDict(best_model_wts, 'fcn16_best')
        
        if epoch % 20 == 19:
            SaveDict(model.state_dict(), dict_name+'_epoch'+str(epoch))
        print()
    
    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    print(model.load_state_dict(best_model_wts))
    return best_model_wts

In [7]:
global M 
M = {'train' : [], 'val' : []}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

fcn16 = FCN16()

# Initialize with fcn32
fcn32dict = LoadDict('fcn32_best')
del fcn32dict['up_final.weight']
fcn16.load_state_dict(fcn32dict, strict = False)

# Train
fcn16_best = train_model(fcn16, device, train_loader, val_loader, lr = 1e-6, dict_name = 'fcn16', num_epochs=200)

#SaveDict(fcn16_best, 'fcn16_best')
SaveDict(M, 'fcn16_metric')

Epoch 0/199
----------
Batch 0)	0.185	0.929	0.716	0.591	0.749
Batch 1)	0.195	0.926	0.756	0.666	0.794
Batch 2)	0.175	0.934	0.825	0.727	0.815
Batch 3)	0.183	0.928	0.773	0.674	0.796
Batch 4)	0.194	0.926	0.697	0.596	0.800
Batch 5)	0.156	0.942	0.808	0.714	0.826
Batch 6)	0.153	0.943	0.743	0.657	0.839
Batch 7)	0.216	0.919	0.743	0.644	0.774
Batch 8)	0.204	0.923	0.793	0.680	0.769
Batch 9)	0.170	0.937	0.805	0.701	0.795
Batch 10)	0.153	0.943	0.792	0.704	0.829
Batch 11)	0.186	0.929	0.797	0.692	0.781
Batch 12)	0.205	0.924	0.775	0.679	0.800
Batch 13)	0.189	0.931	0.774	0.681	0.819
Batch 14)	0.141	0.947	0.810	0.710	0.797
Batch 15)	0.174	0.936	0.753	0.649	0.793
Batch 16)	0.157	0.945	0.771	0.678	0.810
Batch 17)	0.172	0.938	0.793	0.701	0.829
Batch 18)	0.184	0.930	0.802	0.682	0.812
Batch 19)	0.159	0.942	0.800	0.700	0.834
Batch 20)	0.182	0.936	0.811	0.710	0.803
Batch 21)	0.231	0.913	0.725	0.623	0.780
Batch 22)	0.156	0.943	0.771	0.680	0.835
Batch 23)	0.183	0.932	0.773	0.691	0.817
Batch 24)	0.148	0.947	0.778

Val 199)	0.051	0.990	0.500	0.456	0.833
Val 299)	0.025	0.991	0.840	0.687	0.959
Val 399)	0.272	0.891	0.821	0.623	0.541
Val 499)	0.259	0.940	0.360	0.305	0.844
Val 599)	0.013	0.995	0.893	0.632	0.983
Val 699)	0.108	0.960	0.946	0.820	0.897
Val 799)	0.196	0.954	0.962	0.887	0.887
Val 899)	0.175	0.920	0.625	0.542	0.796
Val 999)	0.205	0.936	0.947	0.687	0.542
Val 1099)	0.188	0.909	0.863	0.671	0.744
Val metric:	[0.16843227929587398, 0.9390879918106491, 0.8358647310017564, 0.7031684372832642, 0.8150429559686443]

Epoch 3/199
----------
Batch 0)	0.202	0.922	0.791	0.686	0.773
Batch 1)	0.182	0.935	0.754	0.654	0.805
Batch 2)	0.211	0.921	0.832	0.712	0.778
Batch 3)	0.167	0.938	0.824	0.725	0.822
Batch 4)	0.201	0.926	0.756	0.652	0.807
Batch 5)	0.201	0.927	0.747	0.635	0.785
Batch 6)	0.151	0.944	0.771	0.687	0.849
Batch 7)	0.178	0.938	0.813	0.696	0.802
Batch 8)	0.189	0.928	0.804	0.685	0.768
Batch 9)	0.170	0.934	0.768	0.661	0.808
Batch 10)	0.150	0.944	0.817	0.726	0.839
Batch 11)	0.226	0.920	0.776	0.663	0.790
B

Batch 47)	0.195	0.928	0.799	0.702	0.812
Batch 48)	0.176	0.940	0.781	0.688	0.826
Batch 49)	0.148	0.946	0.800	0.688	0.833
Batch 50)	0.192	0.931	0.791	0.676	0.789
Batch 51)	0.178	0.933	0.852	0.748	0.812
Batch 52)	0.164	0.940	0.777	0.676	0.820
Batch 53)	0.165	0.935	0.797	0.684	0.790
Batch 54)	0.184	0.933	0.798	0.706	0.818
Batch 55)	0.150	0.541	0.454	0.387	0.451
Train:	[0.17600100728676474, 0.9348342041737635, 0.7925358939864515, 0.6864261927952888, 0.8064454551751199]
Val 99)	0.400	0.844	0.809	0.627	0.677
Val 199)	0.157	0.945	0.951	0.863	0.846
Val 299)	0.157	0.952	0.953	0.855	0.864
Val 399)	0.204	0.935	0.350	0.237	0.780
Val 499)	0.379	0.842	0.558	0.351	0.480
Val 599)	0.176	0.920	0.625	0.542	0.795
Val 699)	0.035	0.987	0.926	0.773	0.954
Val 799)	0.240	0.905	0.894	0.799	0.777
Val 899)	0.092	0.965	0.960	0.831	0.872
Val 999)	0.130	0.950	0.813	0.687	0.872
Val 1099)	0.106	0.961	0.964	0.810	0.874
Val metric:	[0.16914008827092047, 0.9388516102907329, 0.837351318756651, 0.7030765868765572, 0.8148923

Batch 34)	0.180	0.939	0.824	0.728	0.819
Batch 35)	0.157	0.940	0.836	0.713	0.816
Batch 36)	0.183	0.935	0.791	0.688	0.819
Batch 37)	0.168	0.938	0.786	0.686	0.807
Batch 38)	0.178	0.930	0.823	0.715	0.811
Batch 39)	0.200	0.924	0.779	0.670	0.792
Batch 40)	0.164	0.940	0.829	0.727	0.827
Batch 41)	0.156	0.939	0.838	0.724	0.822
Batch 42)	0.136	0.951	0.800	0.713	0.843
Batch 43)	0.195	0.931	0.776	0.672	0.792
Batch 44)	0.175	0.936	0.768	0.661	0.789
Batch 45)	0.188	0.935	0.745	0.631	0.781
Batch 46)	0.163	0.941	0.755	0.648	0.800
Batch 47)	0.166	0.940	0.827	0.720	0.838
Batch 48)	0.175	0.935	0.792	0.694	0.816
Batch 49)	0.239	0.916	0.718	0.623	0.781
Batch 50)	0.169	0.937	0.851	0.728	0.792
Batch 51)	0.154	0.942	0.868	0.760	0.817
Batch 52)	0.174	0.935	0.845	0.737	0.800
Batch 53)	0.129	0.953	0.763	0.668	0.856
Batch 54)	0.238	0.908	0.771	0.660	0.755
Batch 55)	0.101	0.563	0.484	0.427	0.487
Train:	[0.17398372640927062, 0.9357020806788104, 0.7954259035981449, 0.6890750465257288, 0.8077005494852321]
Val 99)	0.1

Batch 21)	0.184	0.933	0.809	0.694	0.781
Batch 22)	0.175	0.934	0.822	0.706	0.801
Batch 23)	0.175	0.933	0.798	0.677	0.785
Batch 24)	0.160	0.941	0.778	0.673	0.788
Batch 25)	0.166	0.940	0.839	0.706	0.776
Batch 26)	0.223	0.918	0.758	0.650	0.780
Batch 27)	0.234	0.911	0.768	0.644	0.751
Batch 28)	0.198	0.930	0.757	0.664	0.807
Batch 29)	0.205	0.923	0.792	0.674	0.773
Batch 30)	0.147	0.949	0.802	0.659	0.772
Batch 31)	0.179	0.935	0.787	0.673	0.823
Batch 32)	0.149	0.945	0.760	0.640	0.804
Batch 33)	0.190	0.928	0.814	0.709	0.812
Batch 34)	0.155	0.940	0.723	0.613	0.811
Batch 35)	0.159	0.940	0.844	0.736	0.807
Batch 36)	0.127	0.952	0.777	0.680	0.834
Batch 37)	0.223	0.920	0.803	0.689	0.788
Batch 38)	0.190	0.931	0.807	0.703	0.812
Batch 39)	0.144	0.944	0.854	0.750	0.845
Batch 40)	0.196	0.935	0.739	0.646	0.816
Batch 41)	0.194	0.925	0.836	0.738	0.799
Batch 42)	0.148	0.945	0.837	0.741	0.844
Batch 43)	0.166	0.937	0.774	0.651	0.798
Batch 44)	0.207	0.925	0.777	0.675	0.804
Batch 45)	0.196	0.931	0.782	0.684	0.784


...

Batch 46)	0.163	0.943	0.768	0.656	0.776
Batch 47)	0.183	0.932	0.830	0.726	0.803
Batch 48)	0.154	0.947	0.782	0.680	0.819
Batch 49)	0.190	0.927	0.789	0.695	0.792
Batch 50)	0.167	0.939	0.768	0.656	0.795
Batch 51)	0.172	0.934	0.802	0.677	0.784
Batch 52)	0.188	0.929	0.819	0.684	0.763
Batch 53)	0.207	0.927	0.749	0.646	0.804
Batch 54)	0.170	0.938	0.823	0.725	0.817
Batch 55)	0.096	0.564	0.473	0.409	0.492
Train:	[0.1667182005459456, 0.9385209131996003, 0.8021514936406556, 0.6963672065140509, 0.8120869428029845]
Val 99)	0.117	0.950	0.958	0.877	0.863
Val 199)	0.060	0.970	0.934	0.485	0.342
Val 299)	0.093	0.968	0.970	0.821	0.886
Val 399)	0.294	0.886	0.757	0.647	0.761
Val 499)	0.168	0.928	0.624	0.531	0.802
Val 599)	0.148	0.965	0.965	0.921	0.902
Val 699)	0.116	0.953	0.887	0.746	0.863
Val 799)	0.093	0.963	0.731	0.619	0.899
Val 899)	0.105	0.953	0.960	0.815	0.844
Val 999)	0.057	0.978	0.709	0.629	0.915
Val 1099)	0.120	0.949	0.719	0.654	0.888
Val metric:	[0.16111005373545575, 0.9422256939980383, 0.8454197

Batch 33)	0.187	0.934	0.758	0.644	0.789
Batch 34)	0.166	0.938	0.779	0.683	0.817
Batch 35)	0.116	0.956	0.837	0.713	0.825
Batch 36)	0.158	0.942	0.832	0.723	0.836
Batch 37)	0.167	0.936	0.836	0.725	0.816
Batch 38)	0.138	0.951	0.828	0.719	0.837
Batch 39)	0.120	0.957	0.839	0.746	0.856
Batch 40)	0.173	0.937	0.768	0.678	0.830
Batch 41)	0.167	0.936	0.798	0.669	0.780
Batch 42)	0.210	0.921	0.824	0.716	0.784
Batch 43)	0.169	0.935	0.828	0.713	0.777
Batch 44)	0.118	0.958	0.878	0.766	0.825
Batch 45)	0.167	0.941	0.855	0.736	0.798
Batch 46)	0.121	0.955	0.877	0.792	0.865
Batch 47)	0.207	0.925	0.832	0.724	0.792
Batch 48)	0.155	0.942	0.789	0.694	0.840
Batch 49)	0.176	0.935	0.797	0.696	0.806
Batch 50)	0.163	0.938	0.786	0.684	0.809
Batch 51)	0.138	0.949	0.788	0.693	0.845
Batch 52)	0.196	0.928	0.757	0.653	0.798
Batch 53)	0.177	0.940	0.779	0.686	0.831
Batch 54)	0.147	0.943	0.883	0.760	0.809
Batch 55)	0.090	0.568	0.467	0.397	0.473
Train:	[0.16635981689897372, 0.9387739534863783, 0.8026838964673078, 0.697236904

Batch 20)	0.145	0.949	0.807	0.710	0.839
Batch 21)	0.136	0.952	0.862	0.769	0.860
Batch 22)	0.162	0.941	0.794	0.701	0.834
Batch 23)	0.225	0.915	0.725	0.607	0.762
Batch 24)	0.201	0.926	0.769	0.670	0.801
Batch 25)	0.148	0.941	0.872	0.750	0.808
Batch 26)	0.169	0.938	0.799	0.699	0.828
Batch 27)	0.150	0.944	0.821	0.725	0.837
Batch 28)	0.195	0.927	0.777	0.662	0.777
Batch 29)	0.218	0.916	0.795	0.694	0.796
Batch 30)	0.200	0.926	0.794	0.693	0.790
Batch 31)	0.154	0.946	0.809	0.718	0.841
Batch 32)	0.133	0.947	0.844	0.749	0.833
Batch 33)	0.208	0.924	0.809	0.686	0.783
Batch 34)	0.186	0.934	0.782	0.678	0.820
Batch 35)	0.135	0.951	0.807	0.714	0.829
Batch 36)	0.143	0.950	0.766	0.670	0.844
Batch 37)	0.163	0.940	0.809	0.715	0.827
Batch 38)	0.147	0.947	0.790	0.657	0.793
Batch 39)	0.178	0.935	0.869	0.763	0.814
Batch 40)	0.165	0.940	0.772	0.672	0.792
Batch 41)	0.139	0.951	0.752	0.653	0.822
Batch 42)	0.132	0.951	0.846	0.744	0.853
Batch 43)	0.151	0.944	0.819	0.721	0.832
Batch 44)	0.151	0.949	0.819	0.729	0.840


Batch 7)	0.142	0.950	0.827	0.723	0.823
Batch 8)	0.201	0.928	0.797	0.700	0.798
Batch 9)	0.191	0.928	0.786	0.680	0.802
Batch 10)	0.189	0.929	0.711	0.625	0.800
Batch 11)	0.190	0.932	0.853	0.747	0.811
Batch 12)	0.148	0.943	0.827	0.719	0.838
Batch 13)	0.134	0.953	0.880	0.781	0.853
Batch 14)	0.154	0.945	0.737	0.648	0.821
Batch 15)	0.137	0.951	0.811	0.719	0.850
Batch 16)	0.136	0.948	0.802	0.712	0.817
Batch 17)	0.172	0.937	0.808	0.702	0.813
Batch 18)	0.199	0.929	0.738	0.636	0.792
Batch 19)	0.156	0.942	0.848	0.700	0.773
Batch 20)	0.176	0.932	0.842	0.738	0.822
Batch 21)	0.144	0.947	0.837	0.719	0.821
Batch 22)	0.200	0.927	0.804	0.695	0.789
Batch 23)	0.147	0.946	0.853	0.726	0.814
Batch 24)	0.194	0.930	0.787	0.642	0.763
Batch 25)	0.147	0.946	0.806	0.704	0.819
Batch 26)	0.132	0.955	0.817	0.723	0.858
Batch 27)	0.142	0.947	0.840	0.742	0.834
Batch 28)	0.187	0.932	0.779	0.682	0.811
Batch 29)	0.182	0.931	0.862	0.750	0.815
Batch 30)	0.171	0.941	0.784	0.692	0.823
Batch 31)	0.157	0.944	0.789	0.667	0.807
Bat

0